## 1. Execute imports

In [1]:
!pip install sklearn-genetic

You should consider upgrading via the 'c:\users\walter ortiz\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from genetic_selection import GeneticSelectionCV
from sklearn.metrics import classification_report

In [3]:
data_glass = pd.read_csv('glass.csv', delimiter=',')
X = pd.read_csv('glass_formula_b.csv', delimiter=',')
y = data_glass['Type']


In [4]:
X

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,cos(2*pi*(RI-min(RI)+Na-min(Na)))/(max(RI)+max(Na)-min(RI)-min(Na)),...,cos(2*pi*(Si-min(Si)+K-min(K)))/(max(Si)+max(K)-min(Si)-min(K)),cos(2*pi*(Si-min(Si)+Ca-min(Ca)))/(max(Si)+max(Ca)-min(Si)-min(Ca)),cos(2*pi*(Si-min(Si)+Ba-min(Ba)))/(max(Si)+max(Ba)-min(Si)-min(Ba)),cos(2*pi*(Si-min(Si)+Fe-min(Fe)))/(max(Si)+max(Fe)-min(Si)-min(Fe)),cos(2*pi*(K-min(K)+Ca-min(Ca)))/(max(K)+max(Ca)-min(K)-min(Ca)),cos(2*pi*(K-min(K)+Ba-min(Ba)))/(max(K)+max(Ba)-min(K)-min(Ba)),cos(2*pi*(K-min(K)+Fe-min(Fe)))/(max(K)+max(Fe)-min(K)-min(Fe)),cos(2*pi*(Ca-min(Ca)+Ba-min(Ba)))/(max(Ca)+max(Ba)-min(Ca)-min(Ba)),cos(2*pi*(Ca-min(Ca)+Fe-min(Fe)))/(max(Ca)+max(Fe)-min(Ca)-min(Fe)),cos(2*pi*(Ba-min(Ba)+Fe-min(Fe)))/(max(Ba)+max(Fe)-min(Ba)-min(Fe))
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,0.131262,...,8.317420e-02,-0.015201,1.122614e-01,1.607671e-01,-0.042956,0.099335,0.138360,-0.030610,-3.777988e-02,0.273224
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,0.075100,...,-6.850271e-02,-0.026026,1.001493e-01,1.434217e-01,0.042956,-0.105995,-0.147636,-0.058161,-7.178500e-02,0.273224
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,0.050791,...,-7.661533e-02,-0.060042,4.866049e-02,6.968565e-02,-0.003700,-0.082320,-0.114660,-0.042256,-5.215486e-02,0.273224
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,-0.149325,...,-5.796335e-02,-0.051609,3.531623e-02,5.057561e-02,-0.037562,-0.096670,-0.134647,0.017878,2.206654e-02,0.273224
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,-0.143574,...,3.605244e-02,0.051609,-1.432380e-02,-2.051280e-02,0.021693,-0.101609,-0.141526,-0.045825,-5.655936e-02,0.273224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,-0.129031,...,6.172469e-02,-0.058133,7.284846e-02,5.057561e-02,0.028389,0.068101,0.130403,0.026465,-2.391164e-16,0.254037
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,0.050143,...,-8.300901e-17,0.011454,6.123735e-02,-1.604479e-16,0.057884,-0.090206,0.148810,-0.066842,8.715947e-02,-0.230691
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,-0.095888,...,-6.524244e-02,-0.044558,7.559942e-16,-1.261069e-01,0.058811,-0.068101,0.148810,-0.042256,8.855605e-02,-0.174160
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,-0.083955,...,2.616571e-02,0.035928,-7.823395e-02,5.057561e-02,0.056043,-0.096670,0.148810,-0.052406,8.438833e-02,-0.247221


## KNN

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

### Information gain

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'selector__k': ['all']},
                {'classifier': [KNeighborsClassifier()]}]

In [8]:
%%time
knn_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_info.fit(X_train, y_train)

Wall time: 3.65 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001F28005BD30>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [KNeighborsClassifier()]}])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=42)

In [10]:
selector = GeneticSelectionCV(knn_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [11]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.484765  4.94      0.093498]	[ 0.082855  2.540945  0.023581]	[ 0.345098  1.        0.050373]	[ 0.666993  9.        0.159177]
1  	34    	[-799.505294    6.76      800.085826]	[ 2713.077875     2.241071  2712.906685]	[-10000.            1.            0.063506]	[     0.666993     12.        10000.      ]
2  	37    	[-1599.515778     7.5       1600.078163]	[ 3666.271888     1.835756  3666.026443]	[-10000.            2.            0.048189]	[     0.666993     11.        10000.      ]
3  	24    	[-999.453908    7.46     1000.075478]   	[ 3000.182031     1.802332  2999.974841]	[-10000.            3.            0.048189]	[     0.678758     12.        10000.      ]
4  	30    	[-1399.453686     7.94      1400.071606]	[ 3470.090738     1.302459  3469.841424]	[-10000.            5.            0.035741]	[ 

In [12]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI',
 'Mg',
 'Ba',
 'cos(2*pi*(Mg-min(Mg)+K-min(K)))/(max(Mg)+max(K)-min(Mg)-min(K))',
 'cos(2*pi*(Al-min(Al)+Ba-min(Ba)))/(max(Al)+max(Ba)-min(Al)-min(Ba))',
 'cos(2*pi*(K-min(K)+Ba-min(Ba)))/(max(K)+max(Ba)-min(K)-min(Ba))',
 'cos(2*pi*(K-min(K)+Fe-min(Fe)))/(max(K)+max(Fe)-min(K)-min(Fe))']

In [13]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [14]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.52      0.79      0.63        14
           2       0.50      0.33      0.40        15
           3       0.00      0.00      0.00         3
           5       0.40      0.67      0.50         3
           6       1.00      0.50      0.67         2
           7       1.00      0.83      0.91         6

    accuracy                           0.56        43
   macro avg       0.57      0.52      0.52        43
weighted avg       0.56      0.56      0.54        43



In [15]:
selector.n_features_

7

In [11]:
print(cross_val_score(knn_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.5569264069264068


### Variance threshold

In [12]:
from sklearn.feature_selection import VarianceThreshold

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', KNeighborsClassifier())])

search_space = [{'classifier': [KNeighborsClassifier()]}]

In [14]:
%%time
knn_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
knn_variance.fit(X_train, y_train)

Wall time: 98.2 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}],
             scoring='accuracy')

In [19]:
selector = GeneticSelectionCV(knn_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [20]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.473514  5.48      0.115504]	[ 0.08881   2.714701  0.02934 ]	[ 0.285714  1.        0.042013]	[ 0.651429  9.        0.193819]
1  	19    	[-999.507924    7.26     1000.11014 ]	[ 3000.164026     2.364826  2999.963287]	[-10000.            2.            0.076214]	[     0.664286     13.        10000.      ]
2  	26    	[-999.468619    7.72     1000.114281]	[ 3000.177128     2.172924  2999.961907]	[-10000.            3.            0.052778]	[     0.664762     13.        10000.      ]
3  	30    	[-599.399352    7.98      600.118711]	[ 2375.020169     1.516443  2374.838426]	[-10000.            4.            0.089868]	[     0.671905     11.        10000.      ]
4  	28    	[-1399.433705     7.64      1400.100738]	[ 3470.0988       1.797331  3469.829669]	[-10000.            5.            0.078657]	[     0.

In [21]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Mg',
 'Al',
 'Ca',
 'cos(2*pi*(RI-min(RI)+Mg-min(Mg)))/(max(RI)+max(Mg)-min(RI)-min(Mg))',
 'cos(2*pi*(Mg-min(Mg)+Ca-min(Ca)))/(max(Mg)+max(Ca)-min(Mg)-min(Ca))',
 'cos(2*pi*(Mg-min(Mg)+Ba-min(Ba)))/(max(Mg)+max(Ba)-min(Mg)-min(Ba))',
 'cos(2*pi*(Al-min(Al)+K-min(K)))/(max(Al)+max(K)-min(Al)-min(K))']

In [22]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [23]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.66      0.93      0.77        27
           2       0.84      0.73      0.78        22
           3       0.00      0.00      0.00         7
           5       0.67      0.67      0.67         3
           6       0.00      0.00      0.00         2
           7       0.80      0.80      0.80        10

    accuracy                           0.72        71
   macro avg       0.49      0.52      0.50        71
weighted avg       0.65      0.72      0.68        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
print(cross_val_score(knn_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6502164502164501


### Chi-Square

In [16]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', KNeighborsClassifier())])

In [18]:
search_space = [{'classifier': [KNeighborsClassifier()]}]

In [19]:
%%time
knn_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
knn_chi.fit(X_train, y_train)

Wall time: 90.7 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001F2FFE2F1F0>)),
                                       ('classifier', KNeighborsClassifier())]),
             param_grid=[{'classifier': [KNeighborsClassifier()]}])

In [28]:
selector = GeneticSelectionCV(knn_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [29]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.448619  4.8       0.110153]	[ 0.081484  2.529822  0.023383]	[ 0.27381   1.        0.062713]	[ 0.629048  9.        0.168709]
1  	35    	[-1199.543581     6.04      1200.103777]	[ 3249.783906     2.972945  3249.57704 ]	[-10000.            0.            0.077759]	[     0.629048     12.        10000.      ]
2  	29    	[-1199.533467     6.62      1200.10828 ]	[ 3249.787641     2.813468  3249.575377]	[-10000.           1.           0.06708]   	[     0.629048     13.        10000.      ]
3  	31    	[-799.48879     5.74      800.111569]   	[ 2713.082741     3.205059  2712.899093]	[-10000.            1.            0.089842]	[     0.629048     12.        10000.      ]
4  	24    	[-199.438876    3.3       200.114342]   	[ 1400.080162     2.837252  1399.983666]	[-10000.            1.            0.086902]

In [30]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['cos(2*pi*(RI-min(RI)+Ba-min(Ba)))/(max(RI)+max(Ba)-min(RI)-min(Ba))']

In [31]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [32]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.66      0.78      0.71        27
           2       0.62      0.68      0.65        22
           3       0.00      0.00      0.00         7
           5       0.20      0.33      0.25         3
           6       0.00      0.00      0.00         2
           7       0.89      0.80      0.84        10

    accuracy                           0.63        71
   macro avg       0.40      0.43      0.41        71
weighted avg       0.58      0.63      0.60        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
print(cross_val_score(knn_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.5099567099567099


### Logistic Regression

In [21]:
from sklearn.linear_model import LogisticRegression

### Information gain

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', LogisticRegression())])

search_space = [{'selector__k': ['all']},
                {'classifier': [LogisticRegression()]}]

In [24]:
%%time
logistic_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_info.fit(X_train, y_train)

Wall time: 4.14 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001F28005BD30>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [LogisticRegression()]}])

In [37]:
selector = GeneticSelectionCV(logistic_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [38]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.460505  5.24      0.115501]	[ 0.061267  2.159259  0.028094]	[ 0.351429  1.        0.050352]	[ 0.60381   9.        0.172075]
1  	34    	[-999.540981    6.56     1000.109387]	[ 3000.153007     2.475157  2999.963538]	[-10000.            1.            0.060091]	[     0.638571     14.        10000.      ]
2  	28    	[-399.468971    6.46      400.114988]	[ 1959.700191     1.941237  1959.568323]	[-10000.            1.            0.081217]	[     0.651905     13.        10000.      ]
3  	24    	[-799.461524    6.66      800.110246]	[ 2713.090782     1.795661  2712.899484]	[-10000.            4.            0.081217]	[     0.667143     12.        10000.      ]
4  	30    	[-799.438733    7.2       800.111864]	[ 2713.097502     1.788854  2712.899006]	[-10000.            3.            0.065493]	[     0.679

In [39]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Mg',
 'Al',
 'K',
 'Ba',
 'cos(2*pi*(Mg-min(Mg)+K-min(K)))/(max(Mg)+max(K)-min(Mg)-min(K))',
 'cos(2*pi*(Mg-min(Mg)+Ba-min(Ba)))/(max(Mg)+max(Ba)-min(Mg)-min(Ba))',
 'cos(2*pi*(K-min(K)+Ba-min(Ba)))/(max(K)+max(Ba)-min(K)-min(Ba))',
 'cos(2*pi*(K-min(K)+Fe-min(Fe)))/(max(K)+max(Fe)-min(K)-min(Fe))']

In [40]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [41]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.68      0.85      0.75        27
           2       0.67      0.64      0.65        22
           3       0.00      0.00      0.00         7
           5       0.50      0.67      0.57         3
           6       0.00      0.00      0.00         2
           7       0.80      0.80      0.80        10

    accuracy                           0.66        71
   macro avg       0.44      0.49      0.46        71
weighted avg       0.60      0.66      0.63        71



In [25]:
print(cross_val_score(logistic_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.5798701298701298


### Variance threshold

In [26]:
from sklearn.feature_selection import VarianceThreshold

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', LogisticRegression())])

search_space = [{'classifier': [LogisticRegression()]}]

In [28]:
%%time
logistic_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
logistic_variance.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 412 ms


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}],
             scoring='accuracy')

In [45]:
selector = GeneticSelectionCV(logistic_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [46]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.433638  4.08      0.07372 ]	[ 0.066789  2.591062  0.032858]	[ 0.335714  1.        0.041239]	[ 0.580952  9.        0.148632]
1  	28    	[-799.534943    5.94      800.095251]	[ 2713.069132     2.671404  2712.903905]	[-10000.            1.            0.041853]	[     0.636667     12.        10000.      ]
2  	31    	[-799.495705    6.36      800.097158]	[ 2713.080702     2.242855  2712.903343]	[-10000.            3.            0.041853]	[     0.645238     13.        10000.      ]
3  	32    	[-199.423238    6.22      200.108844]	[ 1400.082395     1.64061   1399.984451]	[-10000.            4.            0.065921]	[     0.645238     10.        10000.      ]
4  	36    	[-399.412419    6.5       400.109728]	[ 1959.711734     2.061553  1959.569396]	[-10000.            2.            0.087532]	[     0.651

In [47]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Al',
 'K',
 'Ba',
 'cos(2*pi*(RI-min(RI)+Mg-min(Mg)))/(max(RI)+max(Mg)-min(RI)-min(Mg))',
 'cos(2*pi*(RI-min(RI)+Al-min(Al)))/(max(RI)+max(Al)-min(RI)-min(Al))',
 'cos(2*pi*(Na-min(Na)+Si-min(Si)))/(max(Na)+max(Si)-min(Na)-min(Si))',
 'cos(2*pi*(Mg-min(Mg)+Ba-min(Ba)))/(max(Mg)+max(Ba)-min(Mg)-min(Ba))']

In [48]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [49]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.71      0.93      0.81        27
           2       0.75      0.82      0.78        22
           3       0.00      0.00      0.00         7
           5       1.00      0.67      0.80         3
           6       1.00      0.50      0.67         2
           7       0.89      0.80      0.84        10

    accuracy                           0.76        71
   macro avg       0.73      0.62      0.65        71
weighted avg       0.70      0.76      0.72        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
print(cross_val_score(logistic_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.6402597402597403


### Chi-Square

In [30]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', LogisticRegression())])

In [32]:
search_space = [{'classifier': [LogisticRegression()]}]

In [33]:
%%time
logistic_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
logistic_chi.fit(X_train, y_train)

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

Wall time: 378 ms


C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001F2FFE2F1F0>)),
                                       ('classifier', LogisticRegression())]),
             param_grid=[{'classifier': [LogisticRegression()]}])

In [54]:
selector = GeneticSelectionCV(logistic_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [55]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.442933  4.68      0.107676]	[ 0.063484  2.656614  0.032504]	[ 0.328571  1.        0.041853]	[ 0.580952  9.        0.182267]
1  	36    	[-799.542848    6.8       800.108039]	[ 2713.066801     2.32379   2712.900134]	[-10000.            1.            0.041853]	[     0.580952     12.        10000.      ]
2  	32    	[-1399.550581     7.16      1400.104405]	[ 3470.051644     2.618091  3469.82819 ]	[-10000.            2.            0.032018]	[     0.588095     15.        10000.      ]
3  	34    	[-1199.514914     7.68      1200.10845 ]	[ 3249.794492     1.630215  3249.575314]	[-10000.            4.            0.059448]	[     0.609048     11.        10000.      ]
4  	19    	[-799.478152    7.86      800.110477]   	[ 2713.085878     1.34179   2712.899415]	[-10000.            5.            0.059448]	[ 

In [56]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI',
 'Al',
 'K',
 'cos(2*pi*(RI-min(RI)+K-min(K)))/(max(RI)+max(K)-min(RI)-min(K))',
 'cos(2*pi*(Mg-min(Mg)+K-min(K)))/(max(Mg)+max(K)-min(Mg)-min(K))',
 'cos(2*pi*(Mg-min(Mg)+Ba-min(Ba)))/(max(Mg)+max(Ba)-min(Mg)-min(Ba))']

In [57]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [58]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.62      0.74      0.68        27
           2       0.59      0.77      0.67        22
           3       0.00      0.00      0.00         7
           5       1.00      0.67      0.80         3
           6       1.00      0.50      0.67         2
           7       1.00      0.70      0.82        10

    accuracy                           0.66        71
   macro avg       0.70      0.56      0.61        71
weighted avg       0.63      0.66      0.63        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [34]:
print(cross_val_score(logistic_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_i

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

0.5569264069264068


## SVM

In [35]:
from sklearn.svm import SVC

### Information gain

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', StandardScaler()),
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'selector__k': ['all']},
                {'classifier': [SVC(kernel='linear')]}]

In [38]:
%%time
svm_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_info.fit(X_train, y_train)

Wall time: 3.66 s


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001F28005BD30>)),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [SVC(kernel='linear')]}])

In [63]:
selector = GeneticSelectionCV(svm_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [64]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.471114  5.22      0.116785]	[ 0.068514  2.625186  0.03185 ]	[ 0.340952  1.        0.041853]	[ 0.651429  9.        0.165087]
1  	32    	[-1199.547229     6.78      1200.110931]	[ 3249.782559     2.351935  3249.574398]	[-10000.            2.            0.079598]	[     0.651429     11.        10000.      ]
2  	32    	[-1799.558895     7.7       1800.103934]	[ 3842.08121      2.308679  3841.825847]	[-10000.           2.           0.07894]   	[     0.651429     12.        10000.      ]
3  	40    	[-1799.539505     7.98      1800.104986]	[ 3842.090295     2.024747  3841.825354]	[-10000.            2.            0.071437]	[     0.658095     12.        10000.      ]
4  	35    	[-1799.524514     7.94      1800.103719]	[ 3842.097318     2.101523  3841.825948]	[-10000.            3.            0.083012]

In [65]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Al',
 'K',
 'Fe',
 'cos(2*pi*(RI-min(RI)+Al-min(Al)))/(max(RI)+max(Al)-min(RI)-min(Al))',
 'cos(2*pi*(RI-min(RI)+Si-min(Si)))/(max(RI)+max(Si)-min(RI)-min(Si))',
 'cos(2*pi*(RI-min(RI)+Fe-min(Fe)))/(max(RI)+max(Fe)-min(RI)-min(Fe))',
 'cos(2*pi*(Mg-min(Mg)+Si-min(Si)))/(max(Mg)+max(Si)-min(Mg)-min(Si))',
 'cos(2*pi*(Mg-min(Mg)+Fe-min(Fe)))/(max(Mg)+max(Fe)-min(Mg)-min(Fe))',
 'cos(2*pi*(K-min(K)+Ca-min(Ca)))/(max(K)+max(Ca)-min(K)-min(Ca))']

In [66]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [67]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.62      0.67      0.64        27
           2       0.50      0.73      0.59        22
           3       0.00      0.00      0.00         7
           5       1.00      0.67      0.80         3
           6       0.00      0.00      0.00         2
           7       0.88      0.70      0.78        10

    accuracy                           0.61        71
   macro avg       0.50      0.46      0.47        71
weighted avg       0.56      0.61      0.57        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [39]:
print(cross_val_score(svm_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6080086580086579


### Variance threshold

In [40]:
from sklearn.feature_selection import VarianceThreshold

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', SVC(kernel='linear'))])

search_space = [{'classifier': [SVC(kernel='linear')]}]

In [42]:
%%time
svm_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
svm_variance.fit(X_train, y_train)

Wall time: 118 ms


GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', SVC(kernel='linear'))]),
             param_grid=[{'classifier': [SVC(kernel='linear')]}],
             scoring='accuracy')

In [71]:
selector = GeneticSelectionCV(svm_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [ ]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.


In [ ]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

In [ ]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

In [ ]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

In [43]:
print(cross_val_score(svm_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.6785714285714286


### Chi-Square

In [44]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', SVC())])

In [46]:
search_space = [{'classifier': [SVC()]}]

In [47]:
%%time
svm_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
svm_chi.fit(X_train, y_train)

Wall time: 86.8 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001F2FFE2F1F0>)),
                                       ('classifier', SVC())]),
             param_grid=[{'classifier': [SVC()]}])

In [14]:
selector = GeneticSelectionCV(svm_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [15]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.4698    5.04      0.111182]	[ 0.073841  2.770993  0.03046 ]	[ 0.315238  1.        0.046474]	[ 0.623333  9.        0.162545]
1  	23    	[-999.524629    6.5      1000.099442]	[ 3000.158458     2.609598  2999.966853]	[-10000.            1.            0.061027]	[     0.629048     13.        10000.      ]
2  	34    	[-799.494067    6.86      800.093397]	[ 2713.081185     2.181834  2712.904452]	[-10000.            3.            0.061027]	[     0.629048     12.        10000.      ]
3  	34    	[-799.468667    6.3       800.090987]	[ 2713.088675     2.042058  2712.905163]	[-10000.           2.           0.06369]   	[     0.63     11.    10000.  ]            
4  	25    	[-399.43479     5.9       400.091018]	[ 1959.707168     1.982423  1959.573215]	[-10000.           2.           0.06369]   	[     0.650

In [16]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI',
 'Na',
 'Al',
 'K',
 'Ba',
 'cos(2*pi*(RI-min(RI)+Mg-min(Mg)))/(max(RI)+max(Mg)-min(RI)-min(Mg))',
 'cos(2*pi*(Al-min(Al)+Ba-min(Ba)))/(max(Al)+max(Ba)-min(Al)-min(Ba))']

In [17]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [18]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.96      0.78        27
           2       0.68      0.68      0.68        22
           3       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         3
           6       1.00      0.50      0.67         2
           7       1.00      0.80      0.89        10

    accuracy                           0.70        71
   macro avg       0.56      0.49      0.50        71
weighted avg       0.63      0.70      0.65        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
print(cross_val_score(svm_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.5519480519480519


## Naive Bayes

In [49]:
from sklearn.naive_bayes import GaussianNB

### Information gain

In [50]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([
                 ('selector', SelectKBest(mutual_info_classif, k='all')),
                 ('classifier', GaussianNB())])

search_space = [{'selector__k': ['all']},
                {'classifier': [GaussianNB()]}]

In [52]:
%%time
nb_info = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_info.fit(X_train, y_train)

Wall time: 3.63 s


GridSearchCV(estimator=Pipeline(steps=[('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function mutual_info_classif at 0x000001F28005BD30>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'selector__k': ['all']},
                         {'classifier': [GaussianNB()]}])

In [23]:
selector = GeneticSelectionCV(nb_info,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [24]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.376848  4.4       0.100338]	[ 0.089912  2.135416  0.03149 ]	[ 0.208571  1.        0.036873]	[ 0.539524  9.        0.16686 ]
1  	25    	[-799.588781    5.72      800.10305 ]	[ 2713.053256     2.46609   2712.901606]	[-10000.            1.            0.036873]	[     0.547143     13.        10000.      ]
2  	32    	[-799.568686    6.1       800.107211]	[ 2713.059182     2.147091  2712.900379]	[-10000.            1.            0.041853]	[     0.575238     11.        10000.      ]
3  	29    	[-599.531162    6.12      600.114745]	[ 2374.986868     2.205811  2374.839428]	[-10000.            3.            0.064356]	[     0.575238     12.        10000.      ]
4  	22    	[ 0.534086  5.86      0.121268]      	[ 0.049371  1.886902  0.027225]         	[ 0.292857  3.        0.063154]            	[ 0.63     

In [25]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Al',
 'K',
 'cos(2*pi*(RI-min(RI)+Mg-min(Mg)))/(max(RI)+max(Mg)-min(RI)-min(Mg))',
 'cos(2*pi*(Al-min(Al)+Si-min(Si)))/(max(Al)+max(Si)-min(Al)-min(Si))',
 'cos(2*pi*(Al-min(Al)+K-min(K)))/(max(Al)+max(K)-min(Al)-min(K))',
 'cos(2*pi*(Al-min(Al)+Ba-min(Ba)))/(max(Al)+max(Ba)-min(Al)-min(Ba))',
 'cos(2*pi*(K-min(K)+Ba-min(Ba)))/(max(K)+max(Ba)-min(K)-min(Ba))']

In [26]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [27]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.66      0.93      0.77        27
           2       0.62      0.68      0.65        22
           3       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           7       0.78      0.70      0.74        10

    accuracy                           0.66        71
   macro avg       0.34      0.38      0.36        71
weighted avg       0.55      0.66      0.60        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [53]:
print(cross_val_score(nb_info, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.4813852813852814


### Variance threshold

In [54]:
from sklearn.feature_selection import VarianceThreshold

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('selector', VarianceThreshold()),
                 ('classifier', GaussianNB())])

search_space = [{'classifier': [GaussianNB()]}]

In [56]:
nb_variance = GridSearchCV(pipe, search_space, cv=None, verbose=0, scoring= 'accuracy')
nb_variance.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('selector', VarianceThreshold()),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}], scoring='accuracy')

In [31]:
selector = GeneticSelectionCV(nb_variance,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [32]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.370514  4.78      0.09975 ]	[ 0.096921  2.60223   0.02929 ]	[ 0.180476  1.        0.041853]	[ 0.53381   9.        0.188687]
1  	16    	[ 0.462495  3.94      0.106702]	[ 0.055444  2.27517   0.029668]	[ 0.33619   1.        0.059417]	[ 0.581429  9.        0.188687]
2  	29    	[ 0.492962  3.22      0.098106]	[ 0.07047   1.526958  0.023796]	[ 0.195238  1.        0.052702]	[ 0.608571  8.        0.146558]
3  	27    	[ 0.534886  3.74      0.098939]	[ 0.053881  1.368357  0.026963]	[ 0.364286  1.        0.055216]	[ 0.62381   8.        0.157097]
4  	35    	[ 0.57379   4.44      0.102095]	[ 0.052422  1.416474  0.024975]	[ 0.405714  1.        0.057704]	[ 0.651905  7.        0.153227]
5  	22    	[ 0.592924  5.        0.097257]	[ 0.060501  1.442221  0.023711]	[ 0.300476  1.        0.052778]	[ 0.651905  8.  

In [33]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['Na',
 'Al',
 'cos(2*pi*(Mg-min(Mg)+Al-min(Al)))/(max(Mg)+max(Al)-min(Mg)-min(Al))',
 'cos(2*pi*(Mg-min(Mg)+Fe-min(Fe)))/(max(Mg)+max(Fe)-min(Mg)-min(Fe))',
 'cos(2*pi*(K-min(K)+Ba-min(Ba)))/(max(K)+max(Ba)-min(K)-min(Ba))']

In [34]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [35]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.65      0.89      0.75        27
           2       0.64      0.64      0.64        22
           3       0.00      0.00      0.00         7
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         2
           7       0.73      0.80      0.76        10

    accuracy                           0.65        71
   macro avg       0.34      0.39      0.36        71
weighted avg       0.55      0.65      0.59        71



C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [57]:
print(cross_val_score(nb_variance, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.4813852813852814


### Chi-Square

In [58]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('selector', SelectKBest(chi2, k='all')),
                 ('classifier', GaussianNB())])

In [60]:
search_space = [{'classifier': [GaussianNB()]}]

In [61]:
%%time
nb_chi = GridSearchCV(pipe, search_space, cv=None, verbose=0)
nb_chi.fit(X_train, y_train)

Wall time: 56.4 ms


GridSearchCV(estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('selector',
                                        SelectKBest(k='all',
                                                    score_func=<function chi2 at 0x000001F2FFE2F1F0>)),
                                       ('classifier', GaussianNB())]),
             param_grid=[{'classifier': [GaussianNB()]}])

In [40]:
selector = GeneticSelectionCV(nb_chi,
                              cv=10,
                              verbose=2,
                              scoring="accuracy",
                              max_features=9,
                              n_population=50,
                              crossover_proba=0.5,
                              mutation_proba=0.2,
                              n_generations=40,
                              crossover_independent_proba=0.5,
                              mutation_independent_proba=0.05,
                              tournament_size=3,
                              n_gen_no_change=10,
                              caching=True,
                              n_jobs=-1)

In [41]:
selector = selector.fit(X_train, y_train)

Selecting features with genetic algorithm.
gen	nevals	avg                            	std                            	min                            	max                            
0  	50    	[ 0.358686  4.4       0.09594 ]	[ 0.102615  2.253886  0.030252]	[ 0.173333  1.        0.032857]	[ 0.573333  8.        0.16724 ]
1  	30    	[-399.581457    5.26      400.109056]	[ 1959.677231     2.406741  1959.569534]	[-10000.            1.            0.047954]	[     0.573333     11.        10000.      ]
2  	29    	[-399.53879     6.26      400.111463]	[ 1959.68594      2.152301  1959.569042]	[-10000.            1.            0.062277]	[     0.573333     10.        10000.      ]
3  	34    	[-1199.540067     6.9       1200.093247]	[ 3249.785204     1.931321  3249.580928]	[-10000.            3.            0.073446]	[     0.609048     12.        10000.      ]
4  	25    	[-599.468505    6.9       600.097934]   	[ 2375.002697     1.445683  2374.843675]	[-10000.            4.            0.073446]	[    

In [42]:
# get the selected features
cols = X.columns.tolist()
selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
selected_feats

['RI',
 'Al',
 'K',
 'cos(2*pi*(Na-min(Na)+Ba-min(Ba)))/(max(Na)+max(Ba)-min(Na)-min(Ba))',
 'cos(2*pi*(Mg-min(Mg)+K-min(K)))/(max(Mg)+max(K)-min(Mg)-min(K))',
 'cos(2*pi*(Mg-min(Mg)+Ca-min(Ca)))/(max(Mg)+max(Ca)-min(Mg)-min(Ca))',
 'cos(2*pi*(Al-min(Al)+Si-min(Si)))/(max(Al)+max(Si)-min(Al)-min(Si))',
 'cos(2*pi*(Ba-min(Ba)+Fe-min(Fe)))/(max(Ba)+max(Fe)-min(Ba)-min(Fe))']

In [43]:
# train and test
clf = KNeighborsClassifier()
clf.fit(X_train[selected_feats], y_train)

KNeighborsClassifier()

In [44]:
preds = clf.predict(X_test[selected_feats])
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           1       0.62      0.59      0.60        27
           2       0.56      0.82      0.67        22
           3       0.00      0.00      0.00         7
           5       1.00      0.67      0.80         3
           6       1.00      0.50      0.67         2
           7       1.00      0.70      0.82        10

    accuracy                           0.62        71
   macro avg       0.70      0.55      0.59        71
weighted avg       0.62      0.62      0.60        71



In [62]:
print(cross_val_score(nb_chi, X, y, cv=10).mean())

C:\Users\Walter Ortiz\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 9 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


0.4813852813852814
